In [9]:
import pandas as pd
import cv2
import os
from imutils import build_montages
import tqdm

In [14]:
def bestByImg(csvPath):
    df = pd.read_csv(csvPath)
    df['conf'] = df.obj_conf * df.cls_conf

    # Sort the DataFrame by 'conf' column in descending order
    # Use groupby to get the rows with the maximum 'conf' value for each unique 'img'
    result = (df[df.cls_conf!=1]
        .sort_values(by='conf', ascending=False)
        .groupby('img').first().reset_index())

    return result

def buildMontage(df, imgRoot, show=False):
    images = []

    for imgPath in tqdm.tqdm(df.img):
        image = cv2.imread(os.path.join(imgRoot, imgPath.replace('txt', 'jpg')))
        imgRow = df[df['img'] == imgPath]
        # print(int(xmin))
        xmin = int(imgRow.xmin)
        ymin = int(imgRow.ymin)
        xmax = int(imgRow.xmax)
        ymax = int(imgRow.ymax)
        image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (0,255,0), 3)
        images.append(image)

    montages = build_montages(images, (640//3, 640//4), (5,5))

    if show:
        showMontages(montages)

    return montages

def showMontages(montages):
    for montage in montages:
        cv2.imshow('montage', montage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [11]:
csvPath = 'bg_df_detections.csv'
imgRoot = "C:\\Users\\leofi\OneDrive - Universidade de Lisboa\Documents\GitHub\masters\Data\\fc2015_yolo_uc_hard/test/images"

df = bestByImg(csvPath)
montages = buildMontage(df, imgRoot, show=True)

100%|██████████| 1067/1067 [00:07<00:00, 150.08it/s]
